In [1]:
import numpy as np
from nudging.simulation import generate_datasets
from sklearn.linear_model import BayesianRidge, LogisticRegression
from nudging.model import BiRegressor
from nudging.model import ProbModel, MDMModel, PCAModel
from nudging.cate import get_cate_correlations
from nudging.correlation import smooth_data
from scipy.stats import spearmanr
from matplotlib import pyplot as plt
from tqdm import tqdm
from collections import defaultdict
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from copy import deepcopy
from nudging.evaluate_outcome import evaluate_performance

### Generate datasets

In [2]:
np.random.seed(9817274)
n_data = 5000
default_param = {
    "noise_frac": 0.5,
    "n_features_uncorrelated": 5,
    "n_features_correlated": 5,
    "avg_correlation": 0.18,
    "n_rescale": 0,
}

In [3]:
default_datasets = generate_datasets(n_data, **default_param)

### Define models

We define both the probabilistic model and a regression model (t-learner). In this case we use all the features that are available, which is 10 features in total, of which 2 are age and gender.

In [4]:
models = {
    "mdm": MDMModel(BayesianRidge()),
    "prob_log": ProbModel(LogisticRegression()),
    "prob_bay": ProbModel(BayesianRidge()),
    "t-learner": BiRegressor(BayesianRidge()),
    "pca": PCAModel(BayesianRidge()),
}

Ignore the many convergence warnings in the proba method

In [5]:
@ignore_warnings(category=ConvergenceWarning)
def run_model(model, data):
    cate_perf, out_perf = evaluate_performance(model, data)
    return np.mean(cate_perf), np.mean(out_perf)

def get_results(models, datasets):
    results_cate = defaultdict(lambda: [])
    results_out = defaultdict(lambda: [])
    for dataset in tqdm(datasets):
        for name, model in models.items():
            cate_perf, out_perf = run_model(model, dataset)
            results_out[name].append(out_perf)
            results_cate[name].append(cate_perf)
    return results_out, results_cate

### Compute the average spearman-r correlations for the regression model

In [ ]:
default_results = get_results(models, default_datasets)

 24%|██████████████████████████                                                                                    | 1183/5000 [51:45<2:52:34,  2.71s/it]

### Plot correlation of both models against each other

From the plot below, it shows that the difference between the two models is a lot bigger with a higher number of features (~26%).

In [ ]:
print("CATE")
{name: np.mean(x) for name, x in default_results[0].items()}

In [ ]:
print("Outcome")
{name: np.mean(x) for name, x in default_results[1].items()}

In [ ]:
def plot_correlations(attr, n_data_smooth=50):
    if attr == "n_features":
        param = deepcopy(default_param)
        param.pop("n_features_uncorrelated")
        param.pop("n_features_correlated")
        datasets = generate_datasets(n_data, **param)
        all_results = get_results(models, datasets)
    elif attr in default_param:
        param = deepcopy(default_param)
        param.pop(attr)
        datasets = generate_datasets(n_data, **param)
        all_results = get_results(models, datasets)
    else:
        datasets = default_datasets
        all_results = default_results
    x = np.array([data.truth[attr] for data in datasets])
    for channel in [0, 1]:
        results = all_results[channel]
        if channel == 0:
            channel_name = "CATE"
        else:
            channel_name = "Outcome"
        for linear in [True, False]:
            idx = np.where([data.truth["linear"] == linear for data in datasets])[0]
            plt.figure(dpi=150)

            if linear:
                plt.title("Linear")
            else:
                plt.title("Non-Linear")
            for name, res in results.items():
                res = np.array(res)
                xcor = spearmanr(x[idx], res[idx]).correlation
                x_smooth, y_smooth = smooth_data(x[idx], res[idx], n_data=n_data_smooth)
                plt.plot(x_smooth, y_smooth, label=f"{name}: {xcor:.2f}")
            plt.xlabel(attr)
            plt.ylabel(f"{channel_name} correlation")

            plt.legend()
            plt.show()

### Plot correlations as a function of the dataset parameters

Below we show differences between the two algorithms with respect to the properties of the dataset.

First up is the amount of noise added to the feature matrix. Obviously, more noise means worse results. On the other hand, it seems like the t-learner is better at extracting information out of less noisy data.

In [ ]:
plot_correlations("noise_frac", n_data_smooth=30)

From the next plot it seems that the number of samples a reasonably strong influence now.

In [ ]:
plot_correlations("n_samples", n_data_smooth=30)

Next up is the `control_precision` parameter that signifies how much the outcome of the untreated/control group depends on the other features, with higher values indicating stronger relations. It looks like the probabilistic method has more trouble with stronger relations for the control group, but the difference is relatively uniform.

In [ ]:
plot_correlations("control_precision", n_data_smooth=30)

The last parameter to be looked at is `control_unique`. This controls how similar the responses are for the control group and the treatment group. If the parameter is 1, then it means that there is likely to be less correlation between the control and treatment responses. Differences are small, but it seems like the t-learner has a harder time with lower values, while the probabilistic method has a harder time with larger values.

In [ ]:
plot_correlations("control_unique", n_data_smooth=30)

In [ ]:
plot_correlations("n_features", n_data_smooth=30)

In [ ]:
plot_correlations("avg_correlation", n_data_smooth=30)

In [ ]:
plot_correlations("balance", n_data_smooth=30)

In [ ]:
plot_correlations("n_rescale", n_data_smooth=30)

In [ ]:
def split_linear(corr, datasets):
    linear_res = [corr[i] for i in range(len(datasets)) if datasets[i].linear == True]
    non_linear_res = [corr[i] for i in range(len(datasets)) if datasets[i].linear == False]
    return linear_res, non_linear_res

In [ ]:
print("CATE")
{name: [np.mean(x) for x in split_linear(res, default_datasets)] for name, res in default_results[0].items()}

In [ ]:
print("Outcome")
{name: [np.mean(x) for x in split_linear(res, default_datasets)] for name, res in default_results[1].items()}

Above we have split the results in the linear and non-linear datasets for both the probabilistic and the regression model. We find that the regression model is about 15% better for the linear datasets. While both datasets become drastically worse with non-linearity added, the probabilistic method suffers more than the regression model. In this case, the regression model is about 54% better.